In [3]:
from netCDF4 import Dataset
import numpy as np

In [4]:
# MESH VARIABLES
xgrid = 99
ygrid = 99
zgrid = 1

def checkerboard_profile(fx, fy, max_val, min_val):
    xrange, yrange = xgrid, ygrid
    epsilon = 0.001
    Ax, Ay = 1, 1
    
    #print(f'\nUsing fx={fx} and fy={fy}\n')
    k = fx*2*np.pi/xrange #wavenumber 2pi / L
    m = fy*2*np.pi/yrange
    x=np.arange(xrange)
    y=np.arange(yrange)
    X,Y=np.meshgrid(x,y)
    phi=Ax*np.sin(k*X+epsilon)*Ay*np.sin(m*Y+epsilon)

    phi_star = phi.copy()

    phi_star[phi_star > 0] = max_val
    phi_star[phi_star <= 0] = min_val

    mesh = np.zeros((xgrid, ygrid, zgrid))
    mesh[:, :, 0] = phi_star

    # transpose so z, y, x
    mesh = mesh.T

    mesh = np.ma.array(np.array([mesh]), mask=False, dtype='float32')

    return mesh

def create_modified_netcdf():
    # Code via 
    # https://stackoverflow.com/questions/15141563/python-netcdf-making-a-copy-of-all-variables-and-attributes-but-one
    # User Rich Signell

    # Modified by Sam Frederick, October 2022, January 2023
    toexclude = ['E_SO2']
    emiss_rates = {'E_SO2': 15.228, # mol km^-2 hr^-1
                   'E_NO2': 10.872}
    

    src = Dataset("wrfchemi_00z_d01")
    dst = Dataset("wrfchemi_test", "w")

    # copy global attributes all at once via dictionary
    dst.setncatts(src.__dict__)
    # copy dimensions
    for name, dimension in src.dimensions.items():
        dst.createDimension(
            name, (len(dimension) if not dimension.isunlimited() else None))
    # copy all file data except for the excluded
    for name, variable in src.variables.items():
        if name not in toexclude:
            x = dst.createVariable(name, variable.datatype, variable.dimensions)
            dst[name][:] = src[name][:]
            # copy variable attributes all at once via dictionary
            dst[name].setncatts(src[name].__dict__)
        else:
            x = dst.createVariable(name, variable.datatype, variable.dimensions)
            print(f'..assigning checkboard for {name}')
            # assign the profile for all timesteps (assuming time is first dimension)
            for i in range(dst[name].shape[0]):
                dst[name][i] = checkerboard_profile(max_val=emiss_rates[name], min_val=0.01) 
            # copy variable attributes all at once via dictionary
            dst[name].setncatts(src[name].__dict__)
            
    emiss_rates.pop('E_SO2')
    placeholder_variable = src.variables['E_SO2']
    for species, emiss_rate in emiss_rates.items():
        x = dst.createVariable(species, placeholder_variable.datatype, placeholder_variable.dimensions)
        print(f'..assigning checkboard for {species}')
        # assign the profile for all timesteps (assuming time is first dimension)
        for i in range(dst[species].shape[0]):
            dst[species][i] = checkerboard_profile(max_val=emiss_rate, min_val=0.03) 

        # copy variable attributes all at once via dictionary
        dst[species].setncatts(src['E_SO2'].__dict__)
            
    src.close()
    dst.close()
    return

In [5]:
create_modified_netcdf()

..assigning checkboard for E_SO2
..assigning checkboard for E_NO2
